In [1]:
from regelum.scenario import Scenario
from regelum.simulator import CasADi
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../')

# from src.system_temp import HydraulicSystem
# from src.observer_temp import HydraulicObserver
# from src.policy import PDController

In [2]:
from regelum.utils import rg
from regelum.system import CartPole


class CartPole(CartPole):
    _dim_observation = 4
    _dim_state = 4
    _dim_action = 1

    _parameters = {
        "m_c": 0.1,
        "m_p": 2.0,
        "g": 9.81,
        "l": 0.5,
    }
    _action_bounds = [[-800.0, 800.0]]

    def _compute_state_dynamics(self, time, state, inputs):
        Dstate = rg.zeros(
            self.dim_state,
            prototype=(state, inputs),
        )

        mass_cart, mass_pole, grav_const, length_pole = (
            self.parameters["m_c"],
            self.parameters["m_p"],
            self.parameters["g"],
            self.parameters["l"],
        )

        angle = state[0]
        angle_vel = state[2]
        vel = state[3]
        force = inputs[0]
        sin_angle = rg.sin(angle)
        cos_angle = rg.cos(angle)

        #########################
        ## YOUR CODE GOES HERE ##

        force_part = (
            force # state
            + mass_pole*length_pole*angle_vel**2*sin_angle
        )

        Dstate[0] = angle_vel
        Dstate[1] = vel
        Dstate[2] = (
            grav_const*sin_angle*(mass_cart + mass_pole)
            - cos_angle*force_part
        )/(
            4/3*length_pole*(mass_cart + mass_pole)
            - mass_pole*length_pole*cos_angle**2
        )
        Dstate[3] = (
            force_part - 3/8 * mass_pole * grav_const * rg.sin(2*angle)
        )/(
            mass_cart + mass_pole - 3/4 * mass_pole * cos_angle**2
        )

        ## YOUR CODE ENDS HERE ##
        #########################

        return Dstate

    def _get_observation(self, time, state, inputs):
        return state


In [3]:
from regelum.policy import Policy
from regelum.utils import rg
from regelum.model import ModelWeightContainer
from regelum.system import ComposedSystem, System
from regelum.optimizable.core.configs import CasadiOptimizerConfig
from numpy import ndarray
from regelum.model import Model, ModelNN, ModelQuadLin
from regelum.optimizable.core.configs import OptimizerConfig
from regelum.objective import mpc_objective
from regelum.predictor import EulerPredictor
from regelum.objective import RunningObjective
from regelum.data_buffers import DataBuffer

In [4]:
from regelum.callback import (
    ScenarioStepLogger,
    # StateTracker,
    # HistoricalDataCallback,
)
from src.callback import HistoricalDataCallback, SimulatorStepLogger

from regelum import set_ipython_env
%matplotlib inline
callbacks = [SimulatorStepLogger, ScenarioStepLogger, HistoricalDataCallback]
ScenarioStepLogger.cooldown = 0.01
callbacks = set_ipython_env(callbacks=callbacks, interactive=True)

In [6]:
from regelum.model import ModelQuadLin

# from regelum.typing import RgArray
# from regelum.objective import Objective

# class HydraulicObjective(Objective):
#     def __init__(
#         self,
#         system: HydraulicSystemModel
#     ):
#         self.l_crit = system._parameters["l_crit"]
    
#     def __call__(
#         self, 
#         observation: RgArray, 
#         action: RgArray,
#         is_save_batch_format = False,
#     ):
#         # return rg.array(
#         #     [[(observation[:,0] - self.l_crit)**2]],
#         #     prototype=observation
#         # )
        
#         result = (observation[:,0] - self.l_crit)**2
        
#         return result[None, :]
        
#         # return rg.array(
#         #     [result],
#         #     prototype=observation
#         # )

class HydraulicObjectiveModel(ModelQuadLin):
    
    def __init__(
        self,
        system,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        
        # self.l_crit = system._parameters["l_crit"]
    
    def forward(self, inputs, weights=None):
        if weights is None:
            weights = self.weights
            quad_matrix = self._quad_matrix
            linear_coefs = self._linear_coefs
            # if isinstance(inputs, torch.Tensor):
            #     quad_matrix = torch.FloatTensor(quad_matrix).to(inputs.device)
            #     if linear_coefs is not None:
            #         linear_coefs = torch.FloatTensor(linear_coefs).to(inputs.device)
        else:
            quad_matrix, linear_coefs = self.get_quad_lin(weights)

        # substract = rg.zeros(
        #     inputs.shape,
        #     prototype=inputs
        # )
        
        # substract[:,0] = self.l_crit
        
        # inputs -= substract

        return ModelQuadLin.quadratic_linear_form(
            inputs,
            quad_matrix,
            linear_coefs,
        )

In [7]:
MAX_STEP = 1e-7
PREDICTION_HORIZON = 10
FINAL_TIME = 10e-3
SAMPLING_TIME = 1e-3
ACTION_SIZE = 1

pred_step_size = 3 * SAMPLING_TIME

# Define the initial state (initial position of the kinematic point).
p_atm = 1e5
# initial_state = rg.array([1e3, 0, 0, p_atm, p_atm])

model = ModelWeightContainer(
    dim_output=ACTION_SIZE, 
    weights_init=rg.zeros((PREDICTION_HORIZON + 1, ACTION_SIZE), rc_type=rg.CASADI)
)  # A trivial model contains our predicted actions

system = CartPole()
# system = HydraulicSystem(
#     init_state=initial_state
# )
# system_model = HydraulicSystem(
#     init_state=initial_state
# )
# observer = HydraulicObserver(
#     system=system_model,
# )
predictor = EulerPredictor(
    system=system, pred_step_size=pred_step_size
)  # Predictor is used for MPC-prediction
# RUNNING OBJECTIVE
running_objective_model = HydraulicObjectiveModel(
    system=system,
    quad_matrix_type="diagonal", weights=[1, 0, 0, 0, 0]
)  # A quadratic model for running (a.k.a. instant) objective. We penalize our agent for x and y
# running_objective_model = ObjectiveModel(
#     system=system_model,
# )
running_objective = RunningObjective(
    model=running_objective_model
)  # Wrap `running_objective_model` into `RunningObjective` class
# running_objective = HydraulicObjective(
#     system=system_model,
# )  # Wrap `running_objective_model` into `RunningObjective` class

class MPC(Policy):

    def __init__(
        self,
        model: Model | ModelNN = model,
        system: System | ComposedSystem = system,
        action_bounds: list | ndarray | None = system.action_bounds,
        optimizer_config: OptimizerConfig | None = CasadiOptimizerConfig(),
        discount_factor: float | None = 1,
        epsilon_random_parameter: float | None = None,
        prediction_horizon: int = PREDICTION_HORIZON,
        running_objective: RunningObjective = running_objective,
        predictor: EulerPredictor = predictor,
    ):
        """Instantiate MPC policy."""
        super().__init__(
            model,
            system,
            action_bounds,
            optimizer_config,
            discount_factor,
            epsilon_random_parameter,
        )  # Initialize the parent `Policy` class.

        self.prediction_horizon = prediction_horizon
        self.running_objective = running_objective
        self.predictor = predictor
        ###### Define the optimization problem
        self.observation_var = self.create_variable(
            1,  # dimensionality of axis 0
            self.system.dim_observation,  # dimensionality of axis 1
            name="observation",
            is_constant=True,  # is_constant set to `True` as `observation` is a constant parameter of optimization
        )
        self.est_state_var = self.create_variable(
            1,  # dimensionality of axis 0
            self.system.dim_state,  # dimensionality of axis 1
            name="estimated_state",
            is_constant=True,  # is_constant is set to `True` as `estimated_state` is a constant parameter of optimization
        )
        self.policy_model_weights_var = self.create_variable(
            name="policy_model_weights",
            is_constant=False,  # is_constant is set to False because policy_model_weights is a decision variable in our optimization problem
            like=self.model.named_parameters,  # like parameter utilizes the dimensions of the model's weights for compatibility
        )
        ## Let us register bounds for policy model weights to be within action bounds
        (
            self.action_bounds_tiled,
            self.action_initial_guess,
            self.action_min,
            self.action_max,
        ) = self.handle_bounds(
            self.action_bounds,
            self.dim_action,
            tile_parameter=self.model.weights.shape[0],
        )
        self.register_bounds(self.policy_model_weights_var, self.action_bounds_tiled)

        ## Make `Optimizable` aware of objective function and variables it depends on
        self.register_objective(
            self.cost,
            variables=[
                self.observation_var,
                self.est_state_var,
                self.policy_model_weights_var,
            ],
        )

    def optimize(self, databuffer: DataBuffer) -> None:
        """Define optimization routine for `Optimizable` class."""
        new_weights = super().optimize_symbolic(
            **databuffer.get_optimization_kwargs(
                keys=["observation", "estimated_state"],
                optimizer_config=self.optimizer_config,
            ),
            policy_model_weights=self.policy_model_weights_var(),
        )[
            "policy_model_weights"
        ]  # Get the optimized weights from `Optimizable` class
        self.model.update_and_cache_weights(new_weights)

    def cost(self, observation, estimated_state, policy_model_weights):
        """Cost function for MPC Policy."""
        return mpc_objective(
            observation=observation,
            estimated_state=estimated_state,
            policy_model_weights=policy_model_weights,
            discount_factor=self.discount_factor,
            running_objective=self.running_objective,
            prediction_horizon=self.prediction_horizon,
            predictor=self.predictor,
            model=self.model,
        )  # Call `mpc_objective` function to get the cost of current state and sequence of predicted actions

In [8]:
from regelum.critic import CriticTrivial
from regelum.scenario import RLScenario
from regelum.event import Event

initial_state = rg.array([-10, 10, 0, 0])

simulator = CasADi(
    system=system,
    state_init=initial_state,
    action_init=rg.array([0]),
    time_final=FINAL_TIME,
    max_step=MAX_STEP,
    # first_step=1e-6,
    # atol=1e-5,
    # rtol=1e-3,
)

scenario = RLScenario(
    policy=MPC(),
    # observer = observer,
    critic=CriticTrivial(),  # mocked critic
    running_objective=running_objective,
    policy_optimization_event=Event.compute_action,  # run policy optimization on each step
    discount_factor=1.0,
    sampling_time=SAMPLING_TIME,
    simulator=simulator,
    N_episodes=1,
    N_iterations=1,
)

In [9]:
# the output of the cell is omitted
scenario.run()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



[20:22:34] INFO     runn. objective: 100.00, state est.: [-10  10   0   0], observation: [-10  10  ]8;id=865610;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=782604;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    0   0], action: [800.], value: 0.1000, time: 0.0000 (0.0%), episode: 1/1,                      
                    iteration: 1/1                                                                                 

[20:22:35] INFO     runn. objective: 99.99, state est.: [-10.    10.     0.98   0.77],             ]8;id=862012;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=216627;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    observation: [-10.    10.     0.98   0.77], action: [800.], value: 0.2000,                     
                    time: 0.0010 (10.0%), episode: 1/1, iteration: 1/1                                             

[20:22:36] INFO     runn. objective: 99.96, state est.: [-10.    10.     1.97   1.55],             ]8;id=41655;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=299417;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    observation: [-10.    10.     1.97   1.55], action: [800.], value: 0.3000,                     
                    time: 0.0020 (20.0%), episode: 1/1, iteration: 1/1                                             

[20:22:38] INFO     runn. objective: 99.91, state est.: [-10.    10.     2.96   2.33],             ]8;id=17526;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=644463;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    observation: [-10.    10.     2.96   2.33], action: [800.], value: 0.3999,                     
                    time: 0.0030 (30.0%), episode: 1/1, iteration: 1/1                                             

...

[20:22:42] INFO     runn. objective: 99.64, state est.: [-9.98 10.01  6.    4.71], observation:    ]8;id=718440;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=851080;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [-9.98 10.01  6.    4.71], action: [800.], value: 0.6991, time: 0.0060                         
                    (60.0%), episode: 1/1, iteration: 1/1                                                          

[20:22:43] INFO     runn. objective: 99.51, state est.: [-9.98 10.02  7.05  5.53], observation:    ]8;id=302365;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=27980;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [-9.98 10.02  7.05  5.53], action: [800.], value: 0.7986, time: 0.0070                         
                    (70.0%), episode: 1/1, iteration: 1/1                                                          

[20:22:44] INFO     runn. objective: 99.36, state est.: [-9.97 10.03  8.12  6.36], observation:    ]8;id=557228;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=708672;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [-9.97 10.03  8.12  6.36], action: [800.], value: 0.8980, time: 0.0080                         
                    (80.0%), episode: 1/1, iteration: 1/1                                                          

[20:22:46] INFO     runn. objective: 99.19, state est.: [-9.96 10.03  9.22  7.21], observation:    ]8;id=635897;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=693264;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [-9.96 10.03  9.22  7.21], action: [800.], value: 0.9972, time: 0.0090                         
                    (90.0%), episode: 1/1, iteration: 1/1                                                          

[20:22:47] INFO     runn. objective: 99.00, state est.: [-9.95 10.04 10.35  8.08], observation:    ]8;id=341967;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py\callback.py]8;;\:]8;id=46267;file:///Users/vulf/.pyenv/versions/3.11.8/envs/acm-env/lib/python3.11/site-packages/regelum/callback.py#1109\1109]8;;\
                    [-9.95 10.04 10.35  8.08], action: [800.], value: 1.0962, time: 0.0100                         
                    (100.0%), episode: 1/1, iteration: 1/1                                                         